In [2]:
import pandas as pd
import glob
import json
from functools import reduce

In [3]:
def get_outcomes():
    """ Get and return ICD codes """""
    f = open('../icd_codes.json')
    outcomes_ = json.load(f)
    f.close()
    return json.loads(outcomes_[0])

In [8]:
#make a data file for each conditiion type
diagnoses = get_outcomes()
total_df = []
for key in diagnoses:
    file_paths = glob.glob(f"../data/state/{key}*.csv")
    dfs = []
    for file in file_paths:
        df = pd.read_csv(file)
        df = df.iloc[:, -4:]
        dfs.append(df)
    combined_df = pd.concat(dfs, axis=0)
    combined_df[f"{key}_hospitalizations"] = combined_df['hospitalizations']
    combined_df = combined_df[['state','residence_county', 'admission_date', f"{key}_hospitalizations"]]
    combined_df.dropna(subset=['state','residence_county', 'admission_date'], inplace=True)
    total_df.append(combined_df)

In [9]:
for df in total_df:
    print(df.shape[0])

7683310
518347
2611726
1214194
1339902
309681
5394719
1183582
2696324
2486528
2735807
783566
1684440


In [10]:
#make one large dataframe with all conditions
merge_cols = ["state","residence_county", "admission_date"]
merged_df = reduce(lambda left, right: pd.merge(left, right, on=merge_cols, how="outer"), total_df)
merged_df['residence_county'] = merged_df['residence_county'].astype(int).astype(str).str.zfill(3)
print(merged_df.shape) #this is the same as the number of rows in the mental_health query which serves as a check that our merge worked properly

merged_df.to_csv(f"../data/combined_data/all_categories_mental_health_counts_state.csv")

(7683310, 16)
